# Análise da solução do problema

Antes deste, foi feito o processo de ETL, que pode ser visto no link a seguir.

https://github.com/samuelrubert/pucrio-mvp-engenhariadados/blob/main/Notebooks/CCUS_etapa1_ETL.ipynb

Também foi feita a análise de qualidade de dados, que pode ser vista no link a seguir.

https://github.com/samuelrubert/pucrio-mvp-engenhariadados/blob/main/Notebooks/CCUS_etapa2_qualidade.ipynb

A seguir, vamos tentar responder cada uma das perguntas feitas na descrição do trabalho.

## 1. Os investimentos em descarbonização têm aumentado?


In [0]:
%sql
SELECT Announcement AS Ano, round(sum(Estimated_capacity_by_IEA_Mt_CO2_yr), 1) AS `Capacidade Anunciada (Mt CO2/ano)`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Announcement IS NOT NULL
GROUP BY 1
ORDER BY 1

Ano,Capacidade Anunciada (Mt CO2/ano)
1982,0.7
1984,3.5
1991,1.0
1997,3.0
2002,0.7
2006,0.6
2007,0.8
2008,8.8
2009,12.4
2010,9.3


É possível observar uma tendência geral de aumento nos investimentos em descarbonização ao longo dos anos. Até 2018, os valores são relativamente baixos e variam muito, mas a partir desse ano há um aumento significativo e consistente na capacidade de projetos anunciados.

Obs.: considerando que os dados foram atualizados em 15/03/2024, as informações de 2024 são incompletas e devem ser ignoradas.

## 2. Quais investimentos estão previstos para o futuro?


In [0]:
%sql
SELECT 'Até 2023' AS `Período de entrada em operação`, round(sum(Estimated_capacity_by_IEA_Mt_CO2_yr), 1) AS `Capacidade Operacional (Mt CO2/ano)`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Operation <= 2023

UNION

SELECT 'Futuro' AS `Período de entrada em operação`, round(sum(Estimated_capacity_by_IEA_Mt_CO2_yr), 1) AS `Capacidade Operacional (Mt CO2/ano)`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Operation > 2023

ORDER BY 1

Período de entrada em operação,Capacidade Operacional (Mt CO2/ano)
Até 2023,73.5
Futuro,1278.3


Verificamos que há previsão futura de uma capacidade operacional de 1243,6 Mt CO2/ano. Em comparação com o valor operacional até 2023, percebemos que será um crescimento significativo, de aproximadamente 17 vezes.

## 3. Quais são os países que mais têm investido em descarbonização?


In [0]:
%sql
SELECT Country AS `País`, round(sum(Estimated_capacity_by_IEA_Mt_CO2_yr), 1) AS `Capacidade Operacional e/ou Anunciada (Mt CO2/ano)`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Country != 'Unknown'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

País,Capacidade Operacional e/ou Anunciada (Mt CO2/ano)
United States,932.5
United Kingdom,248.1
Canada,185.7
Norway,126.9
Netherlands,86.9
Germany,63.3
Denmark,57.2
Australia,52.3
Germany-Norway,40.0
Belgium-Norway,40.0


Verificando os top 10 países em capacidade total, percebemos que os sete maiores são países do hemisfério norte. É especialmente notável que os investimentos nos Estados Unidos excedem a capacidade somada de todos os outros países listados.

## 4. Quais são as empresas que mais têm investido em descarbonização?


In [0]:
%sql
SELECT DISTINCT split(Partners, ',')[0] AS Empresa, round(sum(Estimated_capacity_by_IEA_Mt_CO2_yr), 1) AS `Capacidade Operacional e/ou Anunciada (Mt CO2/ano)`
FROM dt0051_dev.treinamento.iea_ccus_projects
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

Empresa,Capacidade Operacional e/ou Anunciada (Mt CO2/ano)
Howard Midstream Energy Partners LLC,250.0
Carbon Solutions LLC,120.0
ExxonMobil (formerly Denbury carbon solutions),80.0
1PointFive,64.0
Wintershall Dea,51.3
Fluxys,46.0
Fidelis new energy,41.8
Prenco UK (operator),40.0
TotalEnergies,36.4
Equinor,36.2


A análise desta pergunta trouxe dificuldades, devido ao preenchimento do campo Partners pelos autores ser multivalorado e inconsistente, frequentemente trazendo descrições além dos nomes de empresas.

Apenas por motivo didático, foi feita uma consulta a partir da separação do termo anterior à primeira vírgula. Porém, não é um resultado confiável para análise.

## 5. Em quais setores da indústria os esforços em descarbonização mais têm sido aplicados?


In [0]:
%sql
SELECT 
  Sector AS `Setor`, 
  round(sum(Estimated_capacity_by_IEA_Mt_CO2_yr), 1) AS `Capacidade Operacional e/ou Anunciada (Mt CO2/ano)`,
  round(sum(Estimated_capacity_by_IEA_Mt_CO2_yr) * 100 / sum(sum(Estimated_capacity_by_IEA_Mt_CO2_yr)) OVER (), 1) AS `Percentual do Total`
FROM dt0051_dev.treinamento.iea_ccus_projects
GROUP BY 1
ORDER BY 2 DESC

Setor,Capacidade Operacional e/ou Anunciada (Mt CO2/ano),Percentual do Total
Transport,788.0,35.4
T&S,433.8,19.5
Storage,405.7,18.2
Power and heat,174.7,7.9
Hydrogen or ammonia,108.9,4.9
Natural gas processing/LNG,79.6,3.6
DAC,67.3,3.0
Other fuel transformation,62.1,2.8
Cement,36.3,1.6
Biofuels,34.4,1.5


Pode-se concluir que os setores de transporte e de armazenamento são os setores onde estão mais concentrados os investimentos em CCUS. Considerando as três primeiras linhas do resultado, esses setores concentram juntos 73% da capacidade total prevista.

## 6. Quais as principais tecnologias que estão sendo utilizadas?


A base de dados da IEA não traz informações específicas sobre tecnologias utilizadas em cada projeto, por isso não será possível responder à pergunta original.

Em vez disso, vamos fazer uma avaliação dos tipos de projeto que estão sendo contemplados.

In [0]:
%sql
SELECT 
  Project_type AS `Tipo de Projeto`,
  round(sum(Estimated_capacity_by_IEA_Mt_CO2_yr), 1) AS `Capacidade Operacional e/ou Anunciada (Mt CO2/ano)`,
  round(sum(Estimated_capacity_by_IEA_Mt_CO2_yr) * 100 / sum(sum(Estimated_capacity_by_IEA_Mt_CO2_yr)) OVER (), 1) AS `Percentual do Total`
FROM dt0051_dev.treinamento.iea_ccus_projects
GROUP BY 1
ORDER BY 2 DESC

Tipo de Projeto,Capacidade Operacional e/ou Anunciada (Mt CO2/ano),Percentual do Total
Transport,788.0,35.4
T&S,433.8,19.5
Storage,405.7,18.2
Capture,361.1,16.2
Full chain,220.0,9.9
CCU,16.6,0.7


Como visto na pergunta anterior, os projetos de transporte e de armazenamento são maioria. Mas aqui vemos que projetos apenas de captura têm uma participação relevante, e que projetos que capturam CO2 para uso ainda são pouco proeminentes.

## 7. Qual a quantidade de emissões de carbono que se espera reduzir nos próximos anos?

In [0]:
%sql
SELECT round(sum((greatest(least(Suspension_decommissioning, 2054), 2024) - least(greatest(Operation, 2024), 2054)) * Estimated_capacity_by_IEA_Mt_CO2_yr) / 1000, 1) AS `Captura estimada para os próximos 30 anos (Gt CO2)`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Operation IS NOT NULL
  AND Estimated_capacity_by_IEA_Mt_CO2_yr IS NOT NULL
  AND Project_type IN ('Full chain', 'Capture', 'CCU')

Captura estimada para os próximos 30 anos (Gt CO2)
11.9


Aqui, verificamos qual será o total de CO2 capturado nos próximos 30 anos, com base nos dados disponíveis.

Multiplicamos a capacidade anual do projeto pelo período de 30 anos, considerando a data de entrada em operação caso seja posterior a 2024, e a data de descomissionamento caso exista e seja anterior a 2054. São considerados apenas tipos de projeto que façam captura de carbono, não somente transporte ou armazenamento.

Chegamos ao valor de aproximadamente 12 bilhões de toneladas de CO2 capturadas nesse período. Para comparação, uma pesquisa rápida indica que uma pessoa emite em média 1,6 tonelada de CO2 por ano. Assim, a capacidade calculada dos projetos poderia neutralizar as emissões médias de cerca de 248 milhões de pessoas no período.

# Discussão

A análise dos investimentos globais em descarbonização revela uma tendência ascendente nos esforços para mitigar as mudanças climáticas, especialmente a partir de 2018. Esse aumento significativo nas capacidades anunciadas de captura e armazenamento de carbono (CCUS) indica uma resposta robusta de governos e empresas ao Acordo de Paris e à necessidade urgente de reduzir as emissões de carbono. Observamos que os investimentos futuros prometem um crescimento substancial, com a capacidade operacional prevista para aumentar cerca de 17 vezes em comparação com os valores até 2023. Esse crescimento é impulsionado principalmente por países do hemisfério norte, com os Estados Unidos liderando significativamente em termos de capacidade total. Esta distribuição geográfica dos investimentos destaca as disparidades regionais nos esforços de descarbonização, sugerindo que políticas e recursos variam amplamente entre os diferentes países.

Além disso, a análise do setor industrial e dos tipos de projetos revela que os setores de transporte e armazenamento são os principais focos dos investimentos em CCUS, concentrando 73% da capacidade total prevista. Esta concentração indica que, embora a captura de carbono seja vital, a infraestrutura para transporte e armazenamento também é crucial para o sucesso das iniciativas de descarbonização. Finalmente, a projeção de captura de carbono para os próximos 30 anos estima que poderia neutralizar as emissões médias de cerca de 248 milhões de pessoas. 

Os resultados obtidos sublinham a importância contínua do investimento e desenvolvimento de tecnologias CCUS como uma peça essencial no combate às mudanças climáticas.

# Autoavaliação
O objetivo principal deste trabalho foi obter perspectivas sobre os esforços globais de descarbonização, analisando tendências, investimentos e resultados. Através das consultas SQL realizadas na base de dados da IEA CCUS, foi possível responder a maioria das questões delineadas inicialmente. Conseguimos identificar um aumento nos investimentos em descarbonização, prever um crescimento significativo na capacidade operacional futura, e destacar os países e setores industriais com maiores investimentos.

Uma das principais dificuldades foi a inconsistência na base de dados, especialmente no campo “Partners”, que dificultou a identificação precisa das empresas que mais investem em descarbonização. A falta de detalhes sobre as tecnologias específicas utilizadas em cada projeto também limitou a profundidade da análise. Para superar essas limitações em trabalhos futuros, seria benéfico obter uma base de dados mais padronizada e detalhada, que permita uma análise mais precisa e abrangente. Além disso, a inclusão de outras fontes de dados e a combinação de métodos qualitativos e quantitativos podem enriquecer a análise.

Ainda assim, avalia-se que o objetivo do trabalho foi cumprido, com destaque para o aprendizado sobre a construção de um processo de ETL e de análise de dados no Databricks.